In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import os

In [5]:
os.chdir('/content/drive/My Drive/yolov4/Dataset_and_Annotations/csv_folder')
classes_data = pd.read_csv('class-descriptions-boxable.csv', header = None)
classes_data.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [6]:
classes = ['Computer monitor', 'Mobile phone']

class_string = []
for i in classes:
  req_classes = classes_data.loc[classes_data[1] == i]
  string = req_classes.iloc[0][0]
  print(string)
  class_string.append(string)
print(class_string)


/m/02522
/m/050k8
['/m/02522', '/m/050k8']


In [7]:
annotation_data = pd.read_csv('train-annotations-bbox.csv',usecols=['ImageID','LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])
annotation_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [8]:
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_string)].copy()
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
593,000060e3121c7305,/m/02522,0.428125,0.510625,0.540769,0.618557
1149,0000b9115cdf1e54,/m/050k8,0.095703,0.707031,0.497396,0.949219
2487,00016982f6086d39,/m/050k8,0.000000,0.997222,0.000000,0.998438
3963,0002a1a755d730ca,/m/050k8,0.178125,0.409375,0.339583,0.695833
5408,000384bb6da4764b,/m/050k8,0.695545,0.985148,0.434375,0.803750


In [9]:
# adding new columns
filtered_class_data ['classNumber'] = ''
filtered_class_data ['center x'] = ''
filtered_class_data ['center y'] = ''
filtered_class_data ['width'] = ''
filtered_class_data ['height'] = ''

In [10]:
for i in range(len(class_string)):
  filtered_class_data.loc[filtered_class_data['LabelName'] == class_string[i], 'classNumber'] = i

In [11]:
filtered_class_data['center x'] = (filtered_class_data['XMax'] + filtered_class_data['XMin']) / 2
filtered_class_data['center y'] = (filtered_class_data['YMax'] + filtered_class_data['YMin']) / 2


In [12]:
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax,classNumber,center x,center y,width,height
593,000060e3121c7305,/m/02522,0.428125,0.510625,0.540769,0.618557,0,0.469375,0.579663,,
1149,0000b9115cdf1e54,/m/050k8,0.095703,0.707031,0.497396,0.949219,1,0.401367,0.723307,,
2487,00016982f6086d39,/m/050k8,0.000000,0.997222,0.000000,0.998438,1,0.498611,0.499219,,
3963,0002a1a755d730ca,/m/050k8,0.178125,0.409375,0.339583,0.695833,1,0.293750,0.517708,,
5408,000384bb6da4764b,/m/050k8,0.695545,0.985148,0.434375,0.803750,1,0.840346,0.619062,,


In [13]:
filtered_class_data['width'] = filtered_class_data['XMax'] - filtered_class_data['XMin'] 
filtered_class_data['height'] = filtered_class_data['YMax'] - filtered_class_data['YMin'] 
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax,classNumber,center x,center y,width,height
593,000060e3121c7305,/m/02522,0.428125,0.510625,0.540769,0.618557,0,0.469375,0.579663,0.082500,0.077788
1149,0000b9115cdf1e54,/m/050k8,0.095703,0.707031,0.497396,0.949219,1,0.401367,0.723307,0.611328,0.451823
2487,00016982f6086d39,/m/050k8,0.000000,0.997222,0.000000,0.998438,1,0.498611,0.499219,0.997222,0.998438
3963,0002a1a755d730ca,/m/050k8,0.178125,0.409375,0.339583,0.695833,1,0.293750,0.517708,0.231250,0.356250
5408,000384bb6da4764b,/m/050k8,0.695545,0.985148,0.434375,0.803750,1,0.840346,0.619062,0.289603,0.369375


In [14]:
YOLO_values = filtered_class_data.loc[:, ['ImageID', 'classNumber', 'center x', 'center y', 'width', 'height']].copy()
YOLO_values.head()
len(YOLO_values)

12477

In [17]:
image_path = '/content/drive/My Drive/yolov4/Dataset_and_Annotations/Dataset/train/Computer monitor_Mobile phone'
os.chdir(image_path)

for current_dir, dirs, files in os.walk('.'):
  for f in files:
    if f.endswith('.jpg'):
      image_title =f[:-4]
      YOLO_file = YOLO_values.loc[YOLO_values['ImageID'] == image_title]

      df =  YOLO_file.loc[:, ['classNumber', 'center x', 'center y', 'width', 'height']].copy()
      save_path = image_path + '/' + image_title + '.txt'

      df.to_csv(save_path, header=False, index=False, sep = ' ')